# Time in Range Collection

## Loading relevant libraries

`tidyverse` is an R library that contains core packages used to read, analyze, and plot data

In [ ]:
library(tidyverse)
library(lubridate)

## Dexcom

Given that the exported .csv files for Dexcom and FreeStyle Libre are formatted differently, reading and wrangling must also be carried out differently. Following this, the formula for calculating TIR is the same for both CGMS.

### Defining function

In [ ]:
dexcom <- function(file) {
    suppressWarnings({
        read <- read_csv(file)
        
        # renaming columns for ease of use
        names(read)[8] = 'Glucose_Value'
        names(read)[2] = 'Timestamp'
        names(read)[14] = 'Transmitter_ID'
        names(read)[5] = 'Patient_Info'
        
        # creating string based on username and birthdate for ID so comparisons can be made across timepoints
        info <- filter(read, Patient_Info != 'NA')
        info_string <- info %>% pull(Patient_Info)
        id_string <- paste(info_string, collapse = '')
        id_string
        
        # removing top 11 rows that contain user's name and alert types
        rows <- filter(read, Transmitter_ID != 'NA') %>%
                filter(Glucose_Value != 'NA')
        
        # selecting only timestamp and glucose value columns for calculations
        cols <- select(rows, Timestamp, Glucose_Value)
        
        # pulling vectors from columns
        glucose_vector_str <- cols %>% pull(Glucose_Value)
        glucose_vector <- as.double(glucose_vector_str) # converting to decimals
        
        time_vector <- cols %>% pull(Timestamp)
        minutes <- minutes(time_vector) # number of readings in min
        
        # calculating percentage of sensor readings within 14 days
        # only data from participants with >= 70% will be used
        Sensor <- (length(glucose_vector) / 4032) * 100
        
        # calculations
        Mean <- mean(glucose_vector, na.rm=TRUE) # mean average glucose
        GMI <- 12.71 + 4.70587 * Mean # glucose management indicator (mmol/mol)
        SD <- sd(glucose_vector, na.rm=TRUE) # standard deviation
        CV <- SD / Mean # coefficient of variation
        
        # time in range
        in_range <- sum(glucose_vector >= 3.9 & glucose_vector <= 10.0, na.rm=TRUE)
        TIR <- round(in_range * 1000 / length(minutes)) / 10
        
        # very low
        very_low <- sum(glucose_vector < 3.0, na.rm=TRUE)
        TBR_VL <- round(very_low * 1000 / length(minutes)) / 10
        
        # low
        low <- sum(glucose_vector >= 3.0 & glucose_vector <= 3.8, na.rm=TRUE)
        TBR_L <- round(low * 1000 / length(minutes)) / 10
        
        # high
        high <- sum(glucose_vector >= 10.1 & glucose_vector <= 13.9, na.rm=TRUE)
        TAR_H <- round(high * 1000 / length(minutes)) / 10
        
        # very high
        very_high <- sum(glucose_vector > 13.9, na.rm=TRUE)
        TAR_VH <- round(very_high * 1000 / length(minutes)) / 10
        
        # consolidating relevant values into data frame
        df <- data.frame(id_string, TIR, TBR_VL, TBR_L, TAR_H, TAR_VH, GMI, SD, CV, Sensor)
    })
}

### Applying Function - Waitlist Participants

Calling the `dexcom` function on all .csv files in the Dexcom folder to create a data frame with all data from all participants in the waitlist group who use Dexcom

In the final data frame and .csv file (adding both Dexcom and Freestyle Libre using participants to a single data frame), the participants' `id_string`, i.e. their name and birthday, will not be visible and they will be assigned to a unique ID.

In [ ]:
dex_waitlist <- list.files(path = 'Data/Participants/Waitlist/Dexcom',    
                       pattern = '*.csv', full.names = TRUE) %>% 
  lapply(dexcom) %>%                                           
  bind_rows %>%
    filter(Sensor >= 70.0)
  
dex_waitlist

dex_waitlist_output <- 'Output/dex_waitlist.csv'
write.csv(dex_waitlist, dex_waitlist_output)

### Applying Function - Treatment Participants

Calling the `dexcom` function on all .csv files in the Dexcom folder to create a data frame with all data from all participants in the treatment group who use Dexcom

In the final data frame and .csv file (adding both Dexcom and Freestyle Libre using participants to a single data frame), the participants' `id_string`, i.e. their name and birthday, will not be visible and they will be assigned to a unique ID.

In [ ]:
dex_treatment <- list.files(path = 'Data/Participants/Treatment/Dexcom',    
                       pattern = '*.csv', full.names = TRUE) %>% 
  lapply(dexcom) %>%                                           
  bind_rows %>%
    filter(Sensor >= 70.0)
  
dex_treatment

dex_treatment_output <- 'Output/dex_treatment.csv'
write.csv(dex_treatment, dex_treatment_output)

### Applying Function - Peer Supporters

Calling the `dexcom` function on all .csv files in the Dexcom folder to create a data frame with all data from all peer supporters who use Dexcom

In the final data frame and .csv file (adding both Dexcom and Freestyle Libre using participants to a single data frame), the peer supporters' `id_string`, i.e. their name and birthday, will not be visible and they will be assigned to a unique ID.

In [ ]:
dex_ps <- list.files(path = 'Data/PeerSupporters/Dexcom',    
                       pattern = '*.csv', full.names = TRUE) %>% 
  lapply(dexcom) %>%                                           
  bind_rows %>%
    filter(Sensor >= 70.0)
dex_ps

dex_ps_output <- 'Output/dex_ps.csv'
write.csv(dex_ps, dex_ps_output)

## Freestyle Libre 2

### Defining function

In [ ]:
libre_2 <- function(file, d1, d2, d3, d4, d5, d6, d7, d8, d9, d10, d11, d12, d13, d14) {
    suppressWarnings({
        
        # initial reading to create id
        read_id <- read_csv(file)
        names(read_id)[1] = 'Patient_report'
        names(read_id)[2] = 'Generated_on'
        
        # creating string based on user name and birthdate for id
        info <- filter(read_id, Patient_report != 'FreeStyle Libre 2') %>%
                filter(Patient_report != 'FreeStyle LibreLink') %>%
                filter(Patient_report != 'Device') %>%
                select(Patient_report, Generated_on)
        patient_string <- info %>% pull(Patient_report)
        date_string <- info %>% pull(Generated_on)
        id_string <- paste(patient_string, date_string, collapse = '')
        
        read <- read_csv(file, skip=2)
        
        # renaming columns for ease of modifying
        names(read)[4] = 'Record_Type'
        names(read)[3] = 'Timestamp'
        names(read)[5] = 'Glucose_Value'
        
        # filtering out record type = 1 (user scans themselves) and record type = 6 (NA values)
        filter_record <- read %>%
                         filter(Record_Type != 1) %>%
                         filter(Record_Type != 6)
        
        # separating timestamp into date and time for ease of modifying
        filter_timestamp <- filter_record %>%
                            separate(Timestamp, sep = " ", into = c("Date", "Time"))
        
        # establish range for dataframe to filter through
        string_range <- c(d1, d2, d3, d4, d5, d6, d7, d8, d9, d10, d11, d12, d13, d14)

        filtered_df <- filter_timestamp %>%
                       filter(Date %in% string_range)
        
        # bind created columns 'Date' and 'Time' into single column 'Timestamp' and remove 'Date' and 'Time'
        new_df <- mutate(filtered_df, Timestamp = paste(Date, Time))
        new_df[, -c(3,4)]
        
        # selecting only timestamp and glucose value columns
        cols <- select(new_df, Timestamp, Glucose_Value)
        
        # pulling vectors from columns
        glucose_vector <- cols %>% pull(Glucose_Value)
        
        time_vector <- cols %>% pull(Timestamp)
        time_dttm <- ymd_hms(time_vector)
        minutes <- minutes(time_dttm) # pulling the number of readings (in minutes)
        
        # calculating percentage of sensor usage within 14 days
        Sensor <- (length(glucose_vector) / 1344) * 100
        
        # performing calculations
        Mean <- mean(glucose_vector, na.rm=TRUE) # average glucose
        GMI <- 12.71 + 4.70587 * Mean # glucose management indicator (mmol/mol)
        SD <- sd(glucose_vector, na.rm=TRUE) # standard deviation
        CV <- SD / Mean # coefficient of variation
        
        # time in range
        in_range <- sum(glucose_vector >= 3.9 & glucose_vector <= 10.0, na.rm=TRUE)
        TIR <- round(in_range * 1000 / length(minutes)) / 10
        
        # very low
        very_low <- sum(glucose_vector < 3.0, na.rm=TRUE)
        TBR_VL <- round(very_low * 1000 / length(minutes)) / 10
        
        # low
        low <- sum(glucose_vector <= 3.8 & glucose_vector >= 3.0, na.rm=TRUE)
        TBR_L <- round(low * 1000 / length(minutes)) / 10
        
        # high
        high <- sum(glucose_vector >= 10.1 & glucose_vector <= 13.9, na.rm=TRUE)
        TAR_H <- round(high * 1000 / length(minutes)) / 10
        
        # very high
        very_high <- sum(glucose_vector > 13.9, na.rm=TRUE)
        TAR_VH <- round(very_high * 1000 / length(minutes)) / 10
        
        # data frame
        df <- data.frame(id_string, TIR, TBR_VL, TBR_L, TAR_H, TAR_VH, GMI, SD, CV, Sensor)
    })
}

### Applying Function - Waitlist Participants

In [ ]:
# template: waitlist1 <- libre_2("Data/Participants/Waitlist/Libre2/name_of_csv_file", "d1", "d2", "d3", "d4", "d5", "d6", "d7", "d8", "d9", "d10", "d11", "d12", "d13", "d14")
# date format: DD-MM-YYYY (e.g. 04-01-2024)
# variable assignment convention: waitlist1, waitlist2, waitlist3, ..., waitlistn
waitlist1 <- libre_2(...)

# template: bind_rows(variable_1...variable_n)
# for example, bind_rows(waitlist1, waitlist2, waitlist3...) 
# we are essentially putting all of the individual dataframes we got for each participant into 1 dataframe to be exported
libre2_waitlist <- bind_rows(...) %>%
                   filter(Sensor >= 70.0)
libre2_waitlist

libre2_waitlist_output <- 'Output/libre2_waitlist.csv'
write.csv(libre2_waitlist, libre2_waitlist_output)

### Applying Function - Treatment Participants

In [ ]:
# template: treatment1 <- libre_2("Data/Participants/Treatment/Libre2/name_of_csv_file", "d1", "d2", "d3", "d4", "d5", "d6", "d7", "d8", "d9", "d10", "d11", "d12", "d13", "d14")
# date format: DD-MM-YYYY (e.g. 04-01-2024)
# variable assignment convention: treatment1, treatment2, treatment3, ..., treatmentn
treatment1 <- libre_2(...)

# template: bind_rows(variable_1...variable_n)
# for example, bind_rows(treatment1, treatment2, treatment3...) 
libre2_treatment <- bind_rows(...) %>%
                    filter(Sensor >= 70.0)
libre2_treatment

libre2_treatment_output <- 'Output/libre2_treatment.csv'
write.csv(libre2_treatment, libre2_treatment_output)

### Applying Function - Peer Supporters

In [ ]:
# template: treatment1 <- libre_2("Data/PeerSupporters/Libre2/name_of_csv_file", "d1", "d2", "d3", "d4", "d5", "d6", "d7", "d8", "d9", "d10", "d11", "d12", "d13", "d14")
# date format: DD-MM-YYYY (e.g. 04-01-2024)
# variable assignment convention: treatment1, treatment2, treatment3, ..., treatmentn
ps1 <- libre_2(...)

libre2_ps <- bind_rows(..., ...) %>%
             filter(Sensor >= 70.0)
libre2_ps

libre2_ps_output <- 'Output/libre2_ps.csv'
write.csv(libre2_ps, libre2_ps_output)

## Consolidating Dexcom and Freestyle Libre Data

### Waitlist Participants

Binding the data frames from participants who are Dexcom users and Freestyle Libre 2 users in the waitlist group together into a single data frame, which is written into a .csv file that can be downloaded.

In [ ]:
waitlist_output <- 'Output/waitlist.csv'

waitlist <- rbind(dex_waitlist, libre2_waitlist)

waitlist_id <- transform(waitlist, ID = as.numeric(factor(id_string)))

Time = "Baseline" # change this to "6-Months" when calculating for 6-months
Group = "Waitlist"

waitlist_baseline <- waitlist_id %>% 
                     select(TIR, TBR_VL, TBR_L, TAR_H, TAR_VH, GMI, SD, CV, Sensor, ID) %>%
                     mutate(Time = "Baseline",
                            Group = "Waitlist")
                     arrange(ID)
waitlist_baseline
write.csv(waitlist_baseline, waitlist_output)

### Treatment Participants

Binding the data frames from participants who are Dexcom users and Freestyle Libre 2 users in the treatment group together into a single data frame, which is written into a .csv file that can be downloaded.

In [ ]:
treatment_output <- 'Output/treatment.csv'

treatment <- rbind(dex_treatment, libre2_treatment)

treatment_id <- transform(treatment, ID = as.numeric(factor(id_string)))

Time = "Baseline" # change this to "6-Months" when calculating for 6-months
Group = "Treatment"

treatment_baseline <- treatment_id %>% 
                      select(TIR, TBR_VL, TBR_L, TAR_H, TAR_VH, GMI, SD, CV, Sensor, ID, Time, Group) %>% 
                      arrange(ID)
treatment_baseline
write.csv(treatment_baseline, treatment_output)

### Peer Supporters

Binding the data frames from peer supporters who are Dexcom users and Freestyle Libre 2 users together into a single data frame, which is written into a .csv file that can be downloaded.

In [ ]:
ps_output <- 'Output/ps.csv'

ps <- rbind(dex_ps, libre2_ps)

ps_id <- transform(ps, ID = as.numeric(factor(id_string)))

Time = "Baseline" # change this to "6-Months" when calculating for 6-months
Group = "Peer Supporter"

ps_baseline <- ps_id %>% 
               select(TIR, TBR_VL, TBR_L, TAR_H, TAR_VH, GMI, SD, CV, Sensor, ID) %>%
               mutate(Time, Group) %>%
               arrange(ID)
ps_baseline
write.csv(ps_baseline, ps_output)

## Visualization

Code for visualization should be run once time in range values for both baseline and 6-months have been calculated. 

In [ ]:
# Prepare data for visualization

combine <- rbind(waitlist_baseline, waitlist_post, treatment_baseline, treatment_post, ps_baseline, ps_post)

Period <- factor(combine$Time, levels=c("Baseline", "6-months"))

In [ ]:
options(repr.plot.width = 15, repr.plot.height = 9)

plot <- 
    combine %>%
    ggplot() +
    geom_line(aes(x = Period, y = TIR, group = ID, color = Group), size = 1) +
    geom_point(aes(x = Period, y = TIR, shape = Period, color = Group), size = 5) +
    geom_hline(yintercept = 70, colour = "red", linetype = "twodash") +
    annotate("text", label = "Target", x = 2.5, y = 71, size = 6, colour = "red") +
    scale_color_manual(values=c("#0000FF", "#741b47", "#ffa600")) +
    ggtitle("Time in Range before and after T1D REACHOUT intervention") +
    xlab("Period (Pre and post intervention)") +
    ylab("Time in Range (%)") +
    theme(text = element_text(size = 18, family = "sans"),
          plot.title = element_text(face = "bold"),
          axis.title = element_text(face = "bold")) +
    ggsave("plot.png") # you may need to adjust text size in order to view all text in the saved image
plot